In [19]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import streamlit as st

# Load the data from the csv file, and parse the dates to datetime format
df_TAT = pd.read_csv("Daten CSV.csv", sep = ";", parse_dates=["Erfassung der Anforderung", "Probenverteilung", "Proben auf Strasse", 
    "Zentrifugation", "Analyten werden gemessen", "Resultate werden übermittelt", 
    "Technische Validation", "Medizinische Validation", "Erstellen & Versenden Bericht"
]) 
df_TAT.head()
df_TAT.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Probennummer                   12 non-null     int64         
 1   Analyt                         12 non-null     object        
 2   Erfassung der Anforderung      12 non-null     datetime64[ns]
 3   Probenverteilung               12 non-null     datetime64[ns]
 4   Proben auf Strasse             12 non-null     datetime64[ns]
 5   Zentrifugation                 12 non-null     datetime64[ns]
 6   Analyten werden gemessen       12 non-null     datetime64[ns]
 7   Resultate werden übermittelt   12 non-null     datetime64[ns]
 8   Technische Validation          12 non-null     datetime64[ns]
 9   Medizinische Validation        12 non-null     datetime64[ns]
 10  Erstellen & Versenden Bericht  12 non-null     datetime64[ns]
dtypes: datetime64[ns](9),

In [20]:
# Add 7 minutes to "Zentrifugation" column to create "Zentrifugation Ende"
df_TAT['Zentrifugation Ende'] = df_TAT['Zentrifugation'] + pd.Timedelta(minutes=7)
# Rearrange columns to place "Zentrifugation Ende" after "Zentrifugation"
cols = df_TAT.columns.tolist()
zentrifugation_index = cols.index('Zentrifugation')
cols.insert(zentrifugation_index + 1, cols.pop(cols.index('Zentrifugation Ende')))
df_TAT = df_TAT[cols]
df_TAT.head()


,Probennummer,Analyt,Erfassung der Anforderung,Probenverteilung,Proben auf Strasse,Zentrifugation,Zentrifugation Ende,Analyten werden gemessen,Resultate werden übermittelt,Technische Validation,Medizinische Validation,Erstellen & Versenden Bericht
0,2211334401,Troponin,2024-10-01 08:40:00,2024-10-01 13:30:00,2024-10-01 14:05:00,2024-10-01 14:18:00,2024-10-01 14:25:00,2024-10-01 14:27:00,2024-10-01 15:10:00,2024-10-01 15:28:00,2024-10-01 15:49:00,2024-10-01 16:20:00
1,2211334402,CRP,2024-10-02 09:20:00,2024-10-02 13:40:00,2024-10-02 14:15:00,2024-10-02 14:28:00,2024-10-02 14:35:00,2024-10-02 14:37:00,2024-10-02 15:20:00,2024-10-02 15:38:00,2024-10-02 15:59:00,2024-10-02 16:30:00
2,2211334403,D-Dimer,2024-10-02 10:05:00,2024-10-02 13:55:00,2024-10-02 14:20:00,2024-10-02 14:33:00,2024-10-02 14:40:00,2024-10-02 14:42:00,2024-10-02 15:25:00,2024-10-02 15:43:00,2024-10-02 16:04:00,2024-10-02 16:35:00
3,2211334404,Troponin,2024-10-02 10:50:00,2024-10-02 14:10:00,2024-10-02 14:25:00,2024-10-02 14:38:00,2024-10-02 14:45:00,2024-10-02 14:47:00,2024-10-02 15:30:00,2024-10-02 15:48:00,2024-10-02 16:09:00,2024-10-02 16:40:00
4,2211334405,CRP,2024-10-02 11:35:00,2024-10-02 14:25:00,2024-10-02 14:35:00,2024-10-02 14:48:00,2024-10-02 14:55:00,2024-10-02 14:57:00,2024-10-02 15:40:00,2024-10-02 15:58:00,2024-10-02 16:19:00,2024-10-02 16:50:00


In [21]:
phases = {
    "Präanalytische Phase": ["Erfassung der Anforderung", "Probenverteilung", "Proben auf Strasse","Zentrifugation", "Zentrifugation Ende"],
    "Analytische Phase": ["Analyten werden gemessen", "Resultate werden übermittelt"],
    "Postanalytische Phase": ["Technische Validation","Medizinische Validation", "Erstellen & Versenden Bericht"]
}

# Example of accessing the data structure
for phase, checkpoints in phases.items():
    print(f"{phase}:")
    for checkpoint in checkpoints:
        print(f"  - {checkpoint}")

Präanalytische Phase:
  - Erfassung der Anforderung
  - Probenverteilung
  - Proben auf Strasse
  - Zentrifugation
  - Zentrifugation Ende
Analytische Phase:
  - Analyten werden gemessen
  - Resultate werden übermittelt
Postanalytische Phase:
  - Technische Validation
  - Medizinische Validation
  - Erstellen & Versenden Bericht


In [22]:
for phase, checkpoints in phases.items():
    # Calculate the time difference between "Erfassung der Anforderung" and "Zentrifugation Ende"
    df_TAT[f"{phase}"] = df_TAT[checkpoints[-1]] - df_TAT[checkpoints[0]]

df_TAT


,Probennummer,Analyt,Erfassung der Anforderung,Probenverteilung,Proben auf Strasse,Zentrifugation,Zentrifugation Ende,Analyten werden gemessen,Resultate werden übermittelt,Technische Validation,Medizinische Validation,Erstellen & Versenden Bericht,Präanalytische Phase,Analytische Phase,Postanalytische Phase
0,2211334401,Troponin,2024-10-01 08:40:00,2024-10-01 13:30:00,2024-10-01 14:05:00,2024-10-01 14:18:00,2024-10-01 14:25:00,2024-10-01 14:27:00,2024-10-01 15:10:00,2024-10-01 15:28:00,2024-10-01 15:49:00,2024-10-01 16:20:00,0 days 05:45:00,0 days 00:43:00,0 days 00:52:00
1,2211334402,CRP,2024-10-02 09:20:00,2024-10-02 13:40:00,2024-10-02 14:15:00,2024-10-02 14:28:00,2024-10-02 14:35:00,2024-10-02 14:37:00,2024-10-02 15:20:00,2024-10-02 15:38:00,2024-10-02 15:59:00,2024-10-02 16:30:00,0 days 05:15:00,0 days 00:43:00,0 days 00:52:00
2,2211334403,D-Dimer,2024-10-02 10:05:00,2024-10-02 13:55:00,2024-10-02 14:20:00,2024-10-02 14:33:00,2024-10-02 14:40:00,2024-10-02 14:42:00,2024-10-02 15:25:00,2024-10-02 15:43:00,2024-10-02 16:04:00,2024-10-02 16:35:00,0 days 04:35:00,0 days 00:43:00,0 days 00:52:00
3,2211334404,Troponin,2024-10-02 10:50:00,2024-10-02 14:10:00,2024-10-02 14:25:00,2024-10-02 14:38:00,2024-10-02 14:45:00,2024-10-02 14:47:00,2024-10-02 15:30:00,2024-10-02 15:48:00,2024-10-02 16:09:00,2024-10-02 16:40:00,0 days 03:55:00,0 days 00:43:00,0 days 00:52:00
4,2211334405,CRP,2024-10-02 11:35:00,2024-10-02 14:25:00,2024-10-02 14:35:00,2024-10-02 14:48:00,2024-10-02 14:55:00,2024-10-02 14:57:00,2024-10-02 15:40:00,2024-10-02 15:58:00,2024-10-02 16:19:00,2024-10-02 16:50:00,0 days 03:20:00,0 days 00:43:00,0 days 00:52:00
5,2211334406,D-Dimer,2024-10-02 12:20:00,2024-10-02 14:40:00,2024-10-02 14:50:00,2024-10-02 15:03:00,2024-10-02 15:10:00,2024-10-02 15:12:00,2024-10-02 15:55:00,2024-10-02 16:13:00,2024-10-02 16:34:00,2024-10-02 17:05:00,0 days 02:50:00,0 days 00:43:00,0 days 00:52:00
6,2211334407,Troponin,2024-10-02 13:05:00,2024-10-02 14:55:00,2024-10-02 15:05:00,2024-10-02 15:18:00,2024-10-02 15:25:00,2024-10-02 15:27:00,2024-10-02 16:10:00,2024-10-02 16:28:00,2024-10-02 16:49:00,2024-10-02 17:20:00,0 days 02:20:00,0 days 00:43:00,0 days 00:52:00
7,2211334408,CRP,2024-10-02 13:50:00,2024-10-02 15:10:00,2024-10-02 15:20:00,2024-10-02 15:33:00,2024-10-02 15:40:00,2024-10-02 15:42:00,2024-10-02 16:25:00,2024-10-02 16:43:00,2024-10-02 17:04:00,2024-10-02 17:35:00,0 days 01:50:00,0 days 00:43:00,0 days 00:52:00
8,2211334409,D-Dimer,2024-10-02 14:35:00,2024-10-02 15:25:00,2024-10-02 15:35:00,2024-10-02 15:48:00,2024-10-02 15:55:00,2024-10-02 15:57:00,2024-10-02 16:40:00,2024-10-02 16:58:00,2024-10-02 17:19:00,2024-10-02 17:50:00,0 days 01:20:00,0 days 00:43:00,0 days 00:52:00
9,2211334410,Troponin,2024-10-02 15:20:00,2024-10-02 15:40:00,2024-10-02 15:50:00,2024-10-02 16:03:00,2024-10-02 16:10:00,2024-10-02 16:12:00,2024-10-02 16:55:00,2024-10-02 17:13:00,2024-10-02 17:34:00,2024-10-02 18:05:00,0 days 00:50:00,0 days 00:43:00,0 days 00:52:00
